### 🔗 Mapeador Automático Ticker ↔ CNPJ (B3/CVM)

Este utilitário Python resolve o problema de desconexão de dados entre a **B3** (que opera via Tickers) e a **CVM** (que opera via CNPJ), criando um mapeamento confiável e automatizado sem intervenção manual.

### 🎯 O Problema (O "Elo Perdido")
Sistemas de análise financeira (como o **Aurum**) frequentemente precisam cruzar dados de cotação (B3) com dados fundamentalistas/cadastrais (CVM). No entanto:
* A **B3** fornece o Ticker (ex: `ABEV3`) mas raramente fornece o CNPJ ou a Razão Social completa na API pública.
* A **CVM** fornece o CNPJ e a Razão Social, mas não sabe qual é o Ticker associado.

Este script cria uma "ponte" inteligente utilizando o **Yahoo Finance** para descobrir o nome comercial e algoritmos de **Fuzzy Matching** para vinculá-lo ao CNPJ oficial.

### 🛠️ Como Funciona (Pipeline Lógico)



1.  **Extração B3:** O script consulta a API interna da B3 (`IndexProxy`) para obter a composição atualizada do índice **IBRX-100**.
2.  **Dados Oficiais CVM:** Baixa automaticamente o arquivo `cad_cia_aberta.csv` diretamente do portal de Dados Abertos da CVM.
3.  **Enriquecimento (A Ponte):** Para cada Ticker da B3, o script consulta o `yfinance` para descobrir o "Nome Longo" da empresa (ex: Converte `PETR4` → "Petróleo Brasileiro S.A. - Petrobras").
4.  **Matching Probabilístico:** Utiliza a biblioteca `rapidfuzz` para comparar o nome obtido no Yahoo com a Razão Social da CVM. Se a similaridade for alta (Score > 70), o vínculo é criado.

### 📋 Pré-requisitos

O script requer Python 3.8+ e as seguintes bibliotecas externas:

pip install pandas requests yfinance rapidfuzz urllib3

```mermaid 
graph TD
    %% Nós de Início e Fim
    Start([Início: main]) --> FetchB3
    End([Fim: Salvar CSV e Estatísticas])

    %% ETAPA 1: Aquisição de Dados
    subgraph "1. Aquisição de Dados"
        FetchB3[📡 Fetch API B3: IBRX-100] --> CheckB3{Sucesso?}
        CheckB3 -- Não --> Stop1([Encerrar])
        CheckB3 -- Sim --> FetchCVM

        FetchCVM[🏛️ Fetch Cadastro CVM Web] --> CheckCVM{Sucesso?}
        CheckCVM -- Não --> Stop2([Encerrar])
        CheckCVM -- Sim --> LoadLocal

        %% CORREÇÃO AQUI: Aspas adicionadas ao redor do texto
        LoadLocal["📂 Load Local Fundamentals<br/>(fundamentals_wide.csv)"] --> CheckLocal{Existe?}
        
        CheckLocal -- Sim --> DataReady[Dados Prontos]
        CheckLocal -- Não --> DataReady
    end

    DataReady --> Enrich

    %% ETAPA 2: Enriquecimento
    subgraph "2. Enriquecimento (Yahoo Finance)"
        Enrich[🔍 Enrich Tickers with Names]
        Enrich -->|Busca nome oficial da empresa| TickersEnriched[/DataFrame Enriquecido/]
    end

    TickersEnriched --> MatchLoop

    %% ETAPA 3: Lógica de Matching (O Coração do Script)
    subgraph "3. Matching Otimizado (Iteração por Ticker)"
        MatchLoop[🔄 Loop: Para cada Ticker] --> SearchName{Tem Nome<br>do Yahoo?}
        
        SearchName -- Não --> UseTicker[Usar Ticker como Nome]
        SearchName -- Sim --> CleanName[Limpar Sufixos S.A./PN/ON]
        
        UseTicker --> Step1
        CleanName --> Step1

        %% Prioridade 1: Local
        Step1{1. Busca Local?} -->|Fuzzy Match em df_local| ScoreLocal{Score >= 70?}
        
        ScoreLocal -- Sim --> SetLocal[✅ Definir CNPJ Local]
        SetLocal --> SourceLocal[Source: local_fundamentals]
        
        ScoreLocal -- Não --> Step2

        %% Prioridade 2: CVM Geral
        Step2{2. Busca CVM?} -->|Fuzzy Match em df_cvm| ScoreCVM{Score CVM > Score Local?}
        
        ScoreCVM -- Não --> NoMatch[❌ Sem Match Confiável]
        NoMatch --> SourceNone[Source: none]

        ScoreCVM -- Sim --> SetCVM[✅ Definir CNPJ da CVM]
        
        %% CORREÇÃO AQUI: Aspas adicionadas para segurança
        SetCVM --> CheckCross{"CNPJ existe<br>no Local?"}
        
        CheckCross -- Sim --> SourceVer[Source: cvm_registry_verified]
        CheckCross -- Não --> SourceNew[⚠️ Source: cvm_registry_new]

        %% Saídas do Loop
        SourceLocal --> AppendRow
        SourceNone --> AppendRow
        SourceVer --> AppendRow
        SourceNew --> AppendRow
        
        AppendRow[Adicionar à Lista Final] --> NextTicker{Próximo?}
        NextTicker -- Sim --> MatchLoop
    end

    NextTicker -- Não --> End

In [1]:
import pandas as pd
import requests
import base64
import json
import logging
import time
import urllib3
import io
import yfinance as yf
from pathlib import Path
from rapidfuzz import process, fuzz

# --- Configuração ---
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

In [2]:
# Diretórios
OUTPUT_DIR = Path("../data/dados_mapeamento")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

FUNDAMENTALS_PATH = Path("../data/cvm/final/fundamentals_wide.csv") 

URL_CVM_CADASTRO = "https://dados.cvm.gov.br/dados/CIA_ABERTA/CAD/DADOS/cad_cia_aberta.csv"

In [3]:
def fetch_ibrx100_from_b3_api() -> pd.DataFrame:
    """Busca tickers do IBRX-100 direto da API da B3."""
    logger.info("📡 [B3] Iniciando requisição à API...")
    
    try:
        params = {"language": "pt-br", "pageNumber": 1, "pageSize": 120, "index": "IBXX", "segment": "1"}
        params_b64 = base64.b64encode(json.dumps(params).encode("utf-8")).decode("utf-8")
        url = f"https://sistemaswebb3-listados.b3.com.br/indexProxy/indexCall/GetPortfolioDay/{params_b64}"
        
        urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
        response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"}, timeout=15, verify=False)
        
        if response.status_code != 200: return None
        
        data = response.json()
        results = data.get('results', [])
        
        if not results: return None
        
        df = pd.DataFrame(results)
        
        coluna_ticker = next((col for col in ['codNeg', 'cod', 'acronym', 'asset'] if col in df.columns), None)
        
        if not coluna_ticker:
            logger.error("❌ Coluna de ticker não encontrada no JSON da B3.")
            return None
            
        df_final = df[[coluna_ticker]].rename(columns={coluna_ticker: 'ticker'})
        df_final['ticker'] = df_final['ticker'].str.strip()
        df_final['ticker_yahoo'] = df_final['ticker'] + ".SA"
        
        logger.info(f"✅ [B3] {len(df_final)} ativos recuperados.")
        return df_final

    except Exception as e:
        logger.error(f"❌ Erro B3: {e}")
        return None


In [4]:

def fetch_cvm_registry() -> pd.DataFrame:
    """Baixa e processa o cadastro oficial de CNPJs da CVM."""
    logger.info("🏛️ [CVM] Baixando cadastro oficial de companhias...")
    
    try:
        response = requests.get(URL_CVM_CADASTRO, timeout=30)
        if response.status_code != 200:
            return None
            
        csv_content = io.StringIO(response.content.decode('latin1')) 
        df_cvm = pd.read_csv(csv_content, sep=';', dtype=str)
        
        df_cvm = df_cvm[df_cvm['SIT'] == 'ATIVO']
        
        df_cvm = df_cvm[['CNPJ_CIA', 'DENOM_SOCIAL']].copy()
        df_cvm['nome_limpo'] = df_cvm['DENOM_SOCIAL'].str.upper().str.strip()
        
        return df_cvm
    except Exception as e:
        logger.error(f"❌ Erro CVM: {e}")
        return None


In [5]:

def load_local_fundamentals() -> pd.DataFrame:
    """
    Carrega o arquivo local fundamentals_wide.csv para usar como 
    fonte prioritária de 'Match'.
    """
    logger.info(f"📂 [Local] Carregando dados fundamentais de: {FUNDAMENTALS_PATH}")
    
    if not FUNDAMENTALS_PATH.exists():
        logger.warning(f"⚠️ Arquivo local {FUNDAMENTALS_PATH} não encontrado. Otimização será ignorada.")
        return None
        
    try:
        df_fund = pd.read_csv(
            FUNDAMENTALS_PATH, 
            sep=';', 
            usecols=['CNPJ_CIA', 'DENOM_CIA'],
            encoding='utf-8-sig'
        )
        
        df_fund = df_fund.drop_duplicates(subset=['CNPJ_CIA']).copy()
        df_fund['nome_limpo'] = df_fund['DENOM_CIA'].str.upper().str.strip()
        
        logger.info(f"✅ [Local] {len(df_fund)} empresas únicas carregadas do histórico.")
        return df_fund
        
    except Exception as e:
        logger.error(f"❌ Erro ao ler arquivo local: {e}")
        return None


In [6]:

def enrich_tickers_with_names(df_b3: pd.DataFrame) -> pd.DataFrame:
    """Usa yfinance para descobrir o nome oficial da empresa por trás do ticker."""
    logger.info("🔍 [Enriquecimento] Buscando nomes das empresas via Yahoo Finance...")
    
    names_map = {}
    tickers_list = df_b3['ticker_yahoo'].tolist()
    
    total = len(tickers_list)
    
    tickers_obj = yf.Tickers(" ".join(tickers_list))
    
    for i, ticker in enumerate(tickers_list):
        try:
            info = tickers_obj.tickers[ticker].info
            name = info.get('longName') or info.get('shortName')
            names_map[ticker] = name.upper() if name else None
        except Exception:
            names_map[ticker] = None
        
        if i % 20 == 0:
            logger.info(f"   Processado {i}/{total}...")

    df_b3['nome_yahoo'] = df_b3['ticker_yahoo'].map(names_map)
    
    clean_names = df_b3['nome_yahoo'].str.replace(r'\s(PN|ON|UNIT|N1|N2|NM|S\.A\.|LTDA)$', '', regex=True)
    df_b3['nome_busca'] = clean_names.fillna(df_b3['ticker'])
    
    return df_b3

def match_ticker_cnpj_optimized(df_b3: pd.DataFrame, df_cvm: pd.DataFrame, df_local: pd.DataFrame = None) -> pd.DataFrame:
    """
    Cruza tickers com CNPJs usando uma estratégia em duas etapas:
    1. Prioridade: Busca no arquivo local (fundamentals_wide).
    2. Fallback: Busca no cadastro geral da CVM.
    """
    logger.info("🤝 [Matching] Cruzando bases com OTIMIZAÇÃO LOCAL...")
    
    matches = []
    
    local_names = []
    local_lookup = {}
    if df_local is not None:
        local_names = df_local['nome_limpo'].tolist()
        local_lookup = df_local.set_index('nome_limpo')['CNPJ_CIA'].to_dict()
    
    cvm_names = df_cvm['nome_limpo'].tolist()
    cvm_lookup = df_cvm.set_index('nome_limpo')['CNPJ_CIA'].to_dict()
    
    for _, row in df_b3.iterrows():
        ticker = row['ticker']
        search_name = row['nome_busca']
        
        if not search_name:
            matches.append({'ticker': ticker, 'CNPJ': None, 'match_score': 0, 'source': 'none'})
            continue

        best_name = None
        score = 0
        cnpj = None
        source = 'none'

        if local_names:
            match_local = process.extractOne(search_name, local_names, scorer=fuzz.token_sort_ratio)
            if match_local:
                name_l, score_l, _ = match_local
                if score_l >= 70: 
                    best_name = name_l
                    score = score_l
                    cnpj = local_lookup.get(best_name)
                    source = 'local_fundamentals'
        
        if score < 70:
            match_cvm = process.extractOne(search_name, cvm_names, scorer=fuzz.token_sort_ratio)
            if match_cvm:
                name_c, score_c, _ = match_cvm
                
                if score_c > score:
                    best_name = name_c
                    score = score_c
                    cnpj = cvm_lookup.get(best_name)
                    
                    in_local = cnpj in local_lookup.values()
                    source = 'cvm_registry_verified' if in_local else 'cvm_registry_new'

        matches.append({
            'ticker': ticker,
            'nome_b3_yahoo': search_name,
            'nome_oficial': best_name,
            'CNPJ': cnpj,
            'match_score': score,
            'source': source 
        })
            
    return pd.DataFrame(matches)


In [7]:
def main():
    start_time = time.time()
    
    df_b3 = fetch_ibrx100_from_b3_api()
    if df_b3 is None: return

    df_cvm = fetch_cvm_registry()
    if df_cvm is None: return

    df_local = load_local_fundamentals()

    df_b3_enriched = enrich_tickers_with_names(df_b3)

    df_final = match_ticker_cnpj_optimized(df_b3_enriched, df_cvm, df_local)

    csv_path = OUTPUT_DIR / "mapa_ticker_cnpj_otimizado.csv"
    df_final.to_csv(csv_path, index=False, sep=';', encoding='utf-8-sig')
    
    parquet_path = OUTPUT_DIR / "mapa_ticker_cnpj_automatizado.parquet" 
    df_final.to_parquet(parquet_path, index=False)
    
    print(f"\n💾 Arquivos salvos com sucesso:")
    print(f"   📄 CSV: {csv_path}")
    print(f"   🚀 Parquet: {parquet_path}")
    
    print("\n--- Resultado Final (Amostra) ---")
    try:
        display(df_final.head(15))
    except NameError:
        print(df_final.head(15).to_string())
    
    total = len(df_final)
    encontrados = df_final['CNPJ'].notna().sum()
    
    from_local = len(df_final[df_final['source'] == 'local_fundamentals'])
    from_cvm_ver = len(df_final[df_final['source'] == 'cvm_registry_verified'])
    from_cvm_new = len(df_final[df_final['source'] == 'cvm_registry_new'])

    print(f"\n📊 Estatísticas de Mapeamento:")
    print(f"   Total Tickers: {total}")
    print(f"   Mapeados: {encontrados} ({(encontrados/total)*100:.1f}%)")
    print(f"   --------------------------------")
    print(f"   ✅ Encontrado no Histórico Local: {from_local}")
    print(f"   ✅ Encontrado na CVM (Já existe no local): {from_cvm_ver}")
    print(f"   ⚠️ Encontrado na CVM (Novo/Sem dados locais): {from_cvm_new}")
    
    print(f"⏱️ Tempo total: {time.time() - start_time:.2f} segundos")

if __name__ == "__main__":
    main()

2025-12-14 23:11:14,864 - INFO - 📡 [B3] Iniciando requisição à API...
2025-12-14 23:11:15,538 - INFO - ✅ [B3] 97 ativos recuperados.
2025-12-14 23:11:15,538 - INFO - 🏛️ [CVM] Baixando cadastro oficial de companhias...
2025-12-14 23:11:16,327 - INFO - 📂 [Local] Carregando dados fundamentais de: ..\data\cvm\final\fundamentals_wide.csv
2025-12-14 23:11:16,401 - INFO - ✅ [Local] 729 empresas únicas carregadas do histórico.
2025-12-14 23:11:16,401 - INFO - 🔍 [Enriquecimento] Buscando nomes das empresas via Yahoo Finance...
2025-12-14 23:11:17,654 - INFO -    Processado 0/97...
2025-12-14 23:11:29,304 - INFO -    Processado 20/97...
2025-12-14 23:11:41,734 - INFO -    Processado 40/97...
2025-12-14 23:11:54,238 - INFO -    Processado 60/97...
2025-12-14 23:12:06,753 - INFO -    Processado 80/97...
2025-12-14 23:12:16,342 - INFO - 🤝 [Matching] Cruzando bases com OTIMIZAÇÃO LOCAL...



💾 Arquivos salvos com sucesso:
   📄 CSV: ..\data\dados_mapeamento\mapa_ticker_cnpj_otimizado.csv
   🚀 Parquet: ..\data\dados_mapeamento\mapa_ticker_cnpj_automatizado.parquet

--- Resultado Final (Amostra) ---


,ticker,nome_b3_yahoo,nome_oficial,CNPJ,match_score,source
0,ALOS3,ALLOS,ALLOS S.A.,05.878.397/0001-32,66.666667,cvm_registry_verified
1,ABEV3,AMBEV,AMBEV S.A.,07.526.557/0001-00,66.666667,cvm_registry_verified
2,ANIM3,ÂNIMA HOLDING,R/HOLDINGS S.A.,52.475.549/0001-36,64.285714,cvm_registry_new
3,ASAI3,SENDAS DISTRIBUIDORA,SENDAS DISTRIBUIDORA S.A.,06.057.223/0001-71,88.888889,local_fundamentals
4,AURE3,AUREN ENERGIA,AUREN ENERGIA S.A.,28.594.234/0001-23,83.870968,local_fundamentals
5,AXIA3,AXIA ENERGIA SA,AUREN ENERGIA S.A.,28.594.234/0001-23,72.727273,local_fundamentals
6,AXIA6,AXIA ENERGIA SA,AUREN ENERGIA S.A.,28.594.234/0001-23,72.727273,local_fundamentals
7,AZZA3,AZZAS 2154,AZZAS 2154 S.A.,16.590.234/0001-76,80.000000,local_fundamentals
8,B3SA3,"B3 S.A. - BRASIL, BOLSA, BALCÃO","B3 S.A. - BRASIL, BOLSA, BALCÃO",09.346.601/0001-25,100.000000,local_fundamentals
9,BBSE3,BB SEGURIDADE PARTICIPAÇÕES,BB SEGURIDADE PARTICIPAÇÕES S.A.,17.344.597/0001-94,91.525424,local_fundamentals



📊 Estatísticas de Mapeamento:
   Total Tickers: 97
   Mapeados: 97 (100.0%)
   --------------------------------
   ✅ Encontrado no Histórico Local: 80
   ✅ Encontrado na CVM (Já existe no local): 14
   ⚠️ Encontrado na CVM (Novo/Sem dados locais): 3
⏱️ Tempo total: 61.65 segundos
